In [8]:
import cv2
import numpy as np
from PIL import Image
import os
import random
import json
import ast
import csv
from openpyxl import Workbook
from tqdm import tqdm

## 2. Get ares for new map

In [2]:

# Ścieżki
map_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total\\Maps\\The_Skeld_map_camera.jpg"


# Zadeklaruj rozmiar okna monitora (przykład: full HD)
monitor_width = 2560
monitor_height = 1440


# Globalne zmienne
drawing = False
ix, iy = -1, -1
spawn_areas = []

# Callback myszki
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, map_copy, spawn_areas

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            temp = map_copy.copy()
            # Pokazujemy wszystkie już zapisane prostokąty
            for area in spawn_areas:
                cv2.rectangle(temp, (area["x_min"], area["y_min"]), (area["x_max"], area["y_max"]), (0, 255, 0), 2)
            # Tymczasowy aktualny
            cv2.rectangle(temp, (ix, iy), (x, y), (0, 0, 255), 2)
            cv2.imshow("Map Editor", temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x_min = min(ix, x)
        x_max = max(ix, x)
        y_min = min(iy, y)
        y_max = max(iy, y)
        spawn_areas.append({
            "x_min": x_min,
            "x_max": x_max,
            "y_min": y_min,
            "y_max": y_max
        })
        # Aktualizujemy kopię, żeby prostokąt został
        cv2.rectangle(map_copy, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        print(f"✅ Dodano area: {spawn_areas[-1]}")

# Wczytaj mapę
map_img = cv2.imread(map_path)
map_copy = map_img.copy()


orig_height, orig_width = map_img.shape[:2]

# Oblicz skalę tak, żeby zmieściła się w oknie monitora
scale_w = monitor_width / orig_width
scale_h = monitor_height / orig_height
scale = min(scale_w, scale_h)

new_width = int(orig_width * scale)
new_height = int(orig_height * scale)

# Zmień rozmiar mapy
map_img = cv2.resize(map_img, (new_width, new_height), interpolation=cv2.INTER_AREA)

# Kopia do rysowania
map_copy = map_img.copy()

cv2.namedWindow("Map Editor")
cv2.setMouseCallback("Map Editor", draw_rectangle)

print("▶️ Klikaj i rysuj wiele spawn areas. ESC = zakończ.")

while True:
    cv2.imshow("Map Editor", map_copy)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # ESC
        break

cv2.destroyAllWindows()

# Zapisz wszystkie spawn area do JSON
with open("spawn_areas.json", "w") as f:
    json.dump(spawn_areas, f, indent=4)
print("💾 Zapisano spawn_areas.json")


▶️ Klikaj i rysuj wiele spawn areas. ESC = zakończ.
💾 Zapisano spawn_areas.json


## 4. Create new maps, with not-absolute location

In [10]:
def overlay_image(background, overlay, x, y, scale_factor):
    """Osadza overlay na background w pozycji (x, y) z obsługą alfa."""
    
    
    
    new_w = int(overlay.shape[1] * scale_factor)
    new_h = int(overlay.shape[0] * scale_factor)

    # Użyj cv2.resize
    overlay = cv2.resize(overlay, (new_w, new_h), interpolation=cv2.INTER_LINEAR)


    bh, bw = background.shape[:2]
    oh, ow = overlay.shape[:2]

    if x + ow > bw or y + oh > bh:
        print("Ostrzeżenie: overlay wykracza poza obraz tła!")
       # print(x+ow, y+oh)
        return background

    # Wyodrębnij kanały
    b, g, r, a = cv2.split(overlay)
    mask = a / 255.0

    for c in range(3):  # RGB
        background[y:y+oh, x:x+ow, c] = (1.0 - mask) * background[y:y+oh, x:x+ow, c] + mask * overlay[:, :, c]

    # Nadpisujemy kanał alfa tła, jeśli potrzebne
    if background.shape[2] == 4:
        background[y:y+oh, x:x+ow, 3] = np.maximum(background[y:y+oh, x:x+ow, 3], a)

    return background
    
def load_json_file(json_path: str):
    """
    Loads a JSON file and returns its contents.

    Args:
        json_path (str): Path to the JSON file.

    Returns:
        object: Parsed JSON data (e.g., list of dicts).
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"✅ Wczytano {len(data)} areas z {json_path}")
    return data
    
def random_coord_in_area_with_size(area):
    """
    Losuje współrzędne w area tak, żeby cały gracz się zmieścił.
    """

    x = random.randint(area["x_min"], area["x_max"])
    y = random.randint(area["y_min"], area["y_max"])
    return (x, y)


def create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc):
    """
    Tworzy obraz postaci z nałożonymi elementami i zwraca go jako numpy array (RGBA).
    
    Args:
        hat_path (str): Ścieżka do pliku czapki.
        mask_path (str): Ścieżka do pliku maski/okularów.
        suit_path (str): Ścieżka do pliku kombinezonu.
        pc_path (str): Ścieżka do bazowej postaci.
    
    Returns:
        np.ndarray: Obraz wynikowy (z kanałem alfa).
    """
    r_pc = random.choice(sprites_pc)
    r_hat = random.choice(sprites_hats)
    r_eye = random.choice(sprites_eyes)
    r_pant = random.choice(sprites_pants)
    
    save_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total"

    # Plik główny (sprite)
    pc_path = "{}\\PC\\{}".format(save_path,  r_pc)
    
    # Dodatkowe elementy
    hat_path = "{}\\Hats\\{}".format(save_path, r_hat)
    mask_path = "{}\\Eyes\\{}".format(save_path,r_eye)
    suit_path = "{}\\Pants\\{}".format(save_path,r_pant)
    # Wczytaj obraz bazowy
   # print(pc_path)
    base_img = cv2.imread(pc_path, cv2.IMREAD_UNCHANGED)
    h, w = base_img.shape[:2]
    
    # Nowe wymiary
    new_w = w + 70 + 70
    new_h = h + 160 + 40

    # Tworzymy nowy obraz (przezroczysty)
    new_img = np.zeros((new_h, new_w, 4), dtype=np.uint8)

    # Wklejamy bazowy obraz na nowe płótno
    new_img[160:160+h, 50:50+w] = base_img

    # Wczytaj elementy
    hat = cv2.imread(hat_path, cv2.IMREAD_UNCHANGED)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    suit = cv2.imread(suit_path, cv2.IMREAD_UNCHANGED)

    # Osadź elementy
    extended_img = overlay_image(new_img, hat, 140, 60, scale_factor=1.5)
    extended_img = overlay_image(extended_img, mask, 200, 180, scale_factor=1.3)
    extended_img = overlay_image(extended_img, suit, 98, 355, scale_factor=2.1)

    return extended_img

def spawn_players_on_map_instant(
        sprites_hats, 
        sprites_eyes, 
        sprites_pants,
        sprites_pc,
        map_img,
        spawn_areas,
        output_image_path,
        num_players=1,
        char_width=100,
        char_height=150,
        scale = 4.4609375
                        ):
    """
    Wstawia graczy na mapę w losowych area.
    Zapisuje nowy obraz oraz plik JSON z koordynatami.
    """

    result_img = map_img.copy()

    # Zmień rozmiar postaci
  

    coords_list = []

        
    area = random.choice(spawn_areas)
    for _ in range(num_players):
        coord = random_coord_in_area_with_size(area)
        x, y = coord
        
        x = int(x * scale)
        y = int(y * scale)

        coords_list.append({"x": x, "y": y})
        char_img = create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc)
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)
        # Wklej gracza na mapę
        if char_img_resized.shape[2] == 4:
            # Kanał alfa
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

            
    # Koordynaty gracza
    player_coords = {'x': x, 'y': y}

    # Rozdzielczość monitora
    screen_size = (2560, 1440)

    img = result_img
    img_h, img_w = img.shape[:2]
    screen_w, screen_h = screen_size
    center_x, center_y = player_coords['x'], player_coords['y']

    # Oblicz granice cropa
    left = max(center_x - screen_w // 2, 0)
    top = max(center_y - screen_h // 2, 0)
    right = left + screen_w
    bottom = top + screen_h

    if right > img_w:
        right = img_w
        left = max(right - screen_w, 0)

    if bottom > img_h:
        bottom = img_h
        top = max(bottom - screen_h, 0)

    # Przytnij
    cropped = img[top:bottom, left:right]

    # Zapisz
    cv2.imwrite(output_image_path, cropped)
  #  print(coords_list)

    return coords_list

def save_coords_to_xlsx(data_list, xlsx_path):
    """
    Zapisuje listę (map_name, list of points) do pliku XLSX.

    Args:
        data_list (list): Lista krotek (map_name, [{'x': x, 'y': y}, ...])
        xlsx_path (str): Ścieżka do pliku XLSX.

    Returns:
        None
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "PlayerCoords"

    # Nagłówki
    ws.append(['map_name', 'point_index', 'x', 'y'])

    for map_name, points in data_list:
        for idx, point in enumerate(points):
            ws.append([map_name, idx, point['x'], point['y']])

    wb.save(xlsx_path)
    print(f"✅ Zapisano dane do {xlsx_path}")
import os
import random
from tqdm import tqdm

def generate_dataset(output_dir, sprites_hats, sprites_eyes, sprites_pants, sprites_pc, map_img, spawn_areas):
    # Podział na katalogi
    subsets = {"train": 0.8, "val": 0.1, "test": 0.1}
    for subset in subsets:
        os.makedirs(os.path.join(output_dir, f"images/{subset}"), exist_ok=True)
        os.makedirs(os.path.join(output_dir, f"labels/{subset}"), exist_ok=True)

    total_empty = 1500
    total_with_players = 4500
    portion = total_with_players // 3

    config = []
    
    def get_subset(idx):
        ratio = idx / (len(config))
        if ratio < subsets["train"]:
            return "train"
        elif ratio < subsets["train"] + subsets["val"]:
            return "val"
        else:
            return "test"
            
    # Dodaj puste screeny
    for i in range(total_empty):
        config.append((i, 0))

    # Dodaj screeny z graczami
    for i in range(total_with_players):
        if i < portion:
            num = 1
        elif i < portion * 2:
            num = 2
        else:
            num = 3
        config.append((total_empty + i, num))

    random.shuffle(config)

    log_path = os.path.join(output_dir, "generation_log.txt")
    with open(log_path, "w") as f, tqdm(total=len(config), desc="Generating dataset") as pbar:
        for idx, num_players in config:
            subset = get_subset(idx)
            img_path = os.path.join(output_dir, f"images/{subset}", f"Map_{idx}.jpg")
            lbl_path = os.path.join(output_dir, f"labels/{subset}", f"Map_{idx}.txt")
    
            result = spawn_players_on_map_instant_4(
                sprites_hats, sprites_eyes, sprites_pants, sprites_pc,
                map_img, spawn_areas, img_path, lbl_path, num_players
            )
    
            f.write(result["log"] + "\n")
            pbar.update(1)


    print(f"✅ Dataset wygenerowany! Obrazy + etykiety w {output_dir}")

def spawn_players_on_map_instant_4(
        sprites_hats, sprites_eyes, sprites_pants, sprites_pc,
        map_img, spawn_areas,
        output_image_path, output_label_path,
        num_players=1,
        char_width=60, char_height=80,
        scale=4.4609375,
        screen_size=(2560, 1440)
    ):

    result_img = map_img.copy()
    spawn_data = []  # <== tu będziemy zbierać oryginalne współrzędne

    # Spawn graczy
    for _ in range(num_players):
        area = random.choice(spawn_areas)
        coord_x, coord_y = random_coord_in_area_with_size(area, char_width, char_height)

        # Skalowanie do rozdzielczości mapy
        x, y = int(coord_x * scale), int(coord_y * scale)

        char_img = create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc)
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)

        # Wklejanie z kanałem alfa
        if char_img_resized.shape[2] == 4:
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s
            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

        # Dodajemy oryginalne dane spawnu
        spawn_data.append({
            "x": x,
            "y": y,
            "width": char_width,
            "height": char_height
        })

    # Kadrowanie widoku
    screen_w, screen_h = screen_size
    img_h, img_w = result_img.shape[:2]
    x, y = random_coord_in_area_with_size(area, char_width, char_height, scale, img_w, img_h)


    if num_players == 0:
        max_left = max(img_w - screen_w, 0)
        max_top = max(img_h - screen_h, 0)
        left = random.randint(0, max_left)
        top = random.randint(0, max_top)
    else:
        min_x = min(s['x'] for s in spawn_data)
        max_x = max(s['x'] for s in spawn_data)
        min_y = min(s['y'] for s in spawn_data)
        max_y = max(s['y'] for s in spawn_data)

        center_x = (min_x + max_x) // 2 + random.randint(-200, 200)
        center_y = (min_y + max_y) // 2 + random.randint(-200, 200)

        left = max(center_x - screen_w // 2, 0)
        top = max(center_y - screen_h // 2, 0)

    right = left + screen_w
    bottom = top + screen_h
    if right > img_w:
        right = img_w
        left = max(right - screen_w, 0)
    if bottom > img_h:
        bottom = img_h
        top = max(bottom - screen_h, 0)

    cropped = result_img[top:bottom, left:right]

    # Augmentacje
    cropped = apply_augmentations(cropped)

    # YOLO labels
    label_lines = []
    for s in spawn_data:
        screen_x = s['x'] - left
        screen_y = s['y'] - top

        x1 = max(0, screen_x)
        y1 = max(0, screen_y)
        x2 = min(screen_w, screen_x + s['width'])
        y2 = min(screen_h, screen_y + s['height'])

        if x2 > x1 and y2 > y1:
            x_center = ((x1 + x2) / 2) / screen_w
            y_center = ((y1 + y2) / 2) / screen_h
            w_norm = (x2 - x1) / screen_w
            h_norm = (y2 - y1) / screen_h

            label_lines.append(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    # Zapis obrazu i labeli
    cv2.imwrite(output_image_path, cropped)
    with open(output_label_path, "w") as f:
        if label_lines:
            f.write("\n".join(label_lines))

    return {
        "log": f"{os.path.basename(output_image_path)} | Players: {num_players} | Labels: {len(label_lines)}",
        "spawn_data": spawn_data  # <== surowe współrzędne
    }



def random_coord_in_area_with_size(area, char_width, char_height):
    """
    Losuje współrzędne w area tak, aby cały gracz zmieścił się w granicach.
    """
    x = random.randint(area["x_min"], area["x_max"] - char_width)
    y = random.randint(area["y_min"], area["y_max"] - char_height)
    return x, y

def apply_augmentations(image):
    """
    Przykładowe augmentacje obrazu (możesz dodać więcej).
    """
    # Przykład: Losowe odbicie w poziomie
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    return image
    
def random_coord_in_area_with_size(area, char_width, char_height, scale=1.0, img_w=None, img_h=None):
    """
    Losuje współrzędne w obszarze area tak, aby cały gracz się zmieścił
    po uwzględnieniu skali i rozmiaru mapy.
    """
    # Minimalny X i Y w skali oryginalnej
    max_x = area["x_max"] - int(char_width / scale)
    max_y = area["y_max"] - int(char_height / scale)

    x = random.randint(area["x_min"], max_x)
    y = random.randint(area["y_min"], max_y)

    x_scaled = int(x * scale)
    y_scaled = int(y * scale)

    # Jeśli podano wymiary mapy, upewnij się, że nie wyjdziemy poza krawędzie
    if img_w is not None:
        x_scaled = min(x_scaled, img_w - char_width)
    if img_h is not None:
        y_scaled = min(y_scaled, img_h - char_height)

    return x_scaled, y_scaled

def spawn_players_on_map_instant_4(
    sprites_hats, sprites_eyes, sprites_pants, sprites_pc,
    map_img, spawn_areas, output_image_path, output_label_path,
    num_players=4, char_width=80, char_height=60, scale=1.0, screen_size=(640, 480)
):
   

    # Kopia mapy
    result_img = map_img.copy()
    img_h, img_w = result_img.shape[:2]

    spawn_data = []

    for _ in range(num_players):
        # Losowanie obszaru i współrzędnych
        area = random.choice(spawn_areas)
        max_x = area["x_max"] - int(char_width / scale)
        max_y = area["y_max"] - int(char_height / scale)

        if max_x <= area["x_min"] or max_y <= area["y_min"]:
            continue  # obszar za mały

        coord_x = random.randint(area["x_min"], max_x)
        coord_y = random.randint(area["y_min"], max_y)

        x = int(coord_x * scale)
        y = int(coord_y * scale)

        # Clamp do granic mapy
        x = min(max(0, x), img_w - char_width)
        y = min(max(0, y), img_h - char_height)

        # Tworzenie postaci
        char_img = create_random_crewmate_return(
            sprites_hats, sprites_eyes, sprites_pants, sprites_pc
        )

        # Skalowanie
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)

        # Jeśli brak kanału alfa – dodaj pełny
        if char_img_resized.shape[2] == 3:
            alpha_channel = np.ones((char_img_resized.shape[0], char_img_resized.shape[1]), dtype=np.uint8) * 255
            char_img_resized = np.dstack((char_img_resized, alpha_channel))

        # Rozdzielenie kanałów
        alpha_s = char_img_resized[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        # Wklejenie z przezroczystością
        for c in range(3):
            result_img[y:y+char_height, x:x+char_width, c] = (
                alpha_s * char_img_resized[:, :, c] +
                alpha_l * result_img[y:y+char_height, x:x+char_width, c]
            )

        # Zapis danych spawnu
        spawn_data.append((x, y, char_width, char_height))

    # Zapisywanie obrazu
    os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
    cv2.imwrite(output_image_path, result_img)

    # Generowanie YOLO labeli
    labels = []
    for (x, y, w, h) in spawn_data:
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h
        labels.append(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    os.makedirs(os.path.dirname(output_label_path), exist_ok=True)
    with open(output_label_path, "w") as f:
        f.write("\n".join(labels))

    return {"log": f"Spawned {len(spawn_data)} players", "labels": labels}



In [11]:
areas = load_json_file("E:\\Project Sherlock\\spawn_areas_camera.json")
# Ścieżki
map_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total\\Maps\\The_Skeld_map_camera.jpg"
map_img = cv2.imread(map_path)

sprites_pc = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\PC")[1:]
sprites_eyes = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\Eyes")[1:]
sprites_pants = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants")[1:]
sprites_hats = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats")[1:]

out_image_coordinates = "E:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\Coordinates_list_per_map.csv"
save_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total"

coordinates_list = []

✅ Wczytano 12 areas z E:\Project Sherlock\spawn_areas_camera.json


In [12]:
spawn_areas=areas

In [13]:
output_images = "E:/Project Sherlock/Grafiki/Grafiki total/Generated_maps"
output_labels = "E:/Project Sherlock/Grafiki/Grafiki total/Generated_labels"

generate_dataset(
    output_dir="E:/Project Sherlock/Grafiki/Grafiki total/dataset - camera",
    sprites_hats=sprites_hats,
    sprites_eyes=sprites_eyes,
    sprites_pants=sprites_pants,
    sprites_pc=sprites_pc,
    map_img=map_img,
    spawn_areas=areas
)


Generating dataset: 100%|██████████████████████████████████████████████████████████| 6000/6000 [02:21<00:00, 42.41it/s]

✅ Dataset wygenerowany! Obrazy + etykiety w E:/Project Sherlock/Grafiki/Grafiki total/dataset - camera
